## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [1]:
import psycopg2 

try:
    %run create_table.py 
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS naturaldisaster_stage(
        year int, 
        seq int, 
        glide varchar,
        disaster_group varchar,
        disaster_subgroup varchar,
        diaster_type varchar, 
        disaster_subtype varchar,
        disaster_subsubtype varchar,
        event_name varchar,
        country varchar,
        iso varchar,
        region varchar,
        continent varchar,
        location varchar(5000),
        origin varchar, 
        associated_dis varchar,
        associated_dis2 varchar,
        OFDA_response varchar, 
        appeal varchar,
        declaration varchar,
        aid_contribution FLOAT,
        dis_mag_value FLOAT,
        dis_mag_scale varchar,
        latitude varchar,
        longitude varchar,
        local_time varchar,
        river_basin varchar(1000),
        start_year BIGINT,
        start_month varchar,
        start_day int,
        end_year BIGINT,
        end_month varchar,
        end_day int,
        total_deaths FLOAT,
 

In [2]:
try:
    %run etl.py 
    
except Exception as e:
    print(e)
    


#### Test tables with some Queries

In [3]:
import pandas as pd
import boto3
import json
import psycopg2

In [6]:
config_conn = configparser.ConfigParser()
config_conn.read('dwh.cfg')

# Run a query

query = ('''
SELECT * 
FROM surfacetemp
''')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config_conn['CLUSTER'].values()))
df = pd.read_sql_query(query, conn)


df.head()

,year,month,country,avg_temperature,avg_temperature_uncertainty
0,1745,2,Åland,-6.575,1.360
1,1748,6,Åland,NaN,NaN
2,1756,10,Åland,6.680,4.140
3,1766,10,Åland,6.512,1.803
4,1772,10,Åland,9.246,2.189


In [10]:
query = ('''
SELECT country, count(*) as num
FROM naturaldisaster_info
GROUP BY country
Order by num desc
''')
df_disnum = pd.read_sql_query(query, conn)

df_disnum

,country,num
0,United States of America (the),1087
1,China,980
2,India,752
3,Philippines (the),668
4,Indonesia,572
...,...,...
223,Bahrain,1
224,United Arab Emirates (the),1
225,Brunei Darussalam,1
226,Qatar,1


In [25]:
query = ('''
SELECT DISTINCT di.disaster_group, sum(dd.total_deaths) as sum_deaths, num(dd.total_damages) as sum_damages
FROM naturaldisaster_damage dd
Join naturaldisaster_info di on dd.country = di.country
GROUP BY disaster_group
Order by sum_deaths desc
''')
df_worstdis = pd.read_sql_query(query, conn)

df_worstdis

DatabaseError: Execution failed on sql '
SELECT di.country, pp.population, count(di.*) as num
FROM naturaldisaster_info di
Join population pp on pp.country = di.country
Order by num desc
': column "di.country" must appear in the GROUP BY clause or be used in an aggregate function


In [38]:
query = ('''
SELECT pp.country, pp.population, di.disaster_group
FROM population pp
Join naturaldisaster_info di on pp.country = di.country
WHERE pp.year = '2000'
Order by pp.population
''')
df_popdis = pd.read_sql_query(query, conn)

df_popdis

,country,population,disaster_group
0,Tuvalu,9394,Natural
1,Tuvalu,9394,Natural
2,Tuvalu,9394,Natural
3,Tuvalu,9394,Natural
4,Tuvalu,9394,Natural
...,...,...,...
11584,China,1262645000,Natural
11585,China,1262645000,Natural
11586,China,1262645000,Natural
11587,China,1262645000,Natural


In [ ]:
conn.close()